## Загрузка и подготовка данных

In [1]:
# Импорт библиотек
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import numpy as np

import warnings

In [2]:
# Устанавливаем необходимые настройки
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
RANDOM_STATE = 123
STATE = np.random.RandomState(123)

In [5]:
def df_info(data):
    # Вывод первых 5 строк датасета
    display(data.head())
    # Таблица корреляции признаков
    display(data[['f0', 'f1', 'f2']].corr())
    # 
    print(data.info())
    print(f'Пропущенных значений в таблице: {data.isna().sum().sum()}')

In [6]:
df_info(first_region)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2
f0,1.000000,-0.440723,-0.003153
f1,-0.440723,1.000000,0.001724
f2,-0.003153,0.001724,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Пропущенных значений в таблице: 0


In [7]:
df_info(second_region)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2
f0,1.000000,0.182287,-0.001777
f1,0.182287,1.000000,-0.002595
f2,-0.001777,-0.002595,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Пропущенных значений в таблице: 0


In [8]:
df_info(third_region)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,f0,f1,f2
f0,1.000000,0.000528,-0.000448
f1,0.000528,1.000000,0.000779
f2,-0.000448,0.000779,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Пропущенных значений в таблице: 0


Избавимся от дубликатов по регионам.

In [9]:
first_region = first_region.drop_duplicates('id')
second_region = second_region.drop_duplicates('id')
third_region = third_region.drop_duplicates('id')

In [10]:
# Функция разделяет входной датасет на выборки с признаками и целевым признаком.
def features_target_split(region):
    features = region.drop(['id', 'product'], axis=1)
    target = region['product']
    return(features, target)

Применим функцию к таблице каждого региона.

In [11]:
fr_features, fr_target = features_target_split(first_region)
sr_features, sr_target = features_target_split(second_region)
tr_features, tr_target = features_target_split(third_region)

**Вывод:** изучена исходная информация, выделены признаки и целевой признак по каждому региону.

## Обучение и проверка модели

In [12]:
def lr_by_reg(features, target):
    # Разделяем выборки с признаками и целевым признаком на обучающие и валидационные
    features_train, features_valid, \
    target_train, target_valid = train_test_split(features, target, train_size=.75, \
                                                  random_state=RANDOM_STATE)
    # Стандартизируем признаки в обучающей и валидационной выборках
    scaler = StandardScaler()
    scaler.fit(features_train)
    columns = features_valid.columns
    features_train[columns] = scaler.transform(features_train[columns])
    features_valid[columns] = scaler.transform(features_valid[columns])
    # Обучаем модель линейной регрессии
    model = LinearRegression()
    model.fit(features_train, target_train)
    # Делаем предсказания, рассчитываем корень от среднеквадратичной ошибки
    # рассчитываем среднее значение предсказанного запаса сырья.
    predictions_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predictions_valid)
    print(f'Средний запас предсказанного сырья: {predictions_valid.mean()}')
    print(f'RMSE модели: {mse ** .5}')
    return(target_valid.reset_index(drop=True), pd.Series(predictions_valid))

Применим функцию для каждого региона.

In [13]:
fr_target_valid, fr_predictions_valid = lr_by_reg(fr_features, fr_target)

Средний запас предсказанного сырья: 92.6806203637552
RMSE модели: 37.595806572623616


In [14]:
sr_target_valid, sr_predictions_valid = lr_by_reg(sr_features, sr_target)

Средний запас предсказанного сырья: 69.31422919477679
RMSE модели: 0.8944711936820202


In [15]:
tr_target_valid, tr_predictions_valid = lr_by_reg(tr_features, tr_target)

Средний запас предсказанного сырья: 94.93789877099006
RMSE модели: 40.04908749269797


**Вывод:** наименьший средний запас предсказанного сырья наблюдается во втором регионе. Но также стоит отметить, что в этом районе корень среднеквадратичной ошибки значительно меньше, чем в других регионах. При предсказании объёма продукта в первом и третьем регионе разброс значения больше, следовательно модель может предсказать убыточную скважину, как прибыльную, либо прибыльную, как убыточную.

## Подготовка к расчёту прибыли

Расчётные показатели для каждого региона.

In [16]:
# Кол-во исследуемых скважин
POINTS_TO_EXPLORE = 500
# Бюджет на разработку скважин
BUDGET = 10000000000
# Планируется разработать скважин на бюджет
POINTS_PER_BUDGET = 200
# Стоимость разработки скважины
COST_PER_POINT = BUDGET / POINTS_PER_BUDGET
# Доход с 1 тысячи баррелей продукта
PRODUCT_PRICE = 450000
# Минимальный объём продукта для покрытия расходов на разработку одной скважины
zero_profit_product = COST_PER_POINT / PRODUCT_PRICE

In [17]:
print(f'Минимальный объём продукта в скважине для безубыточной разработки: {zero_profit_product}')

Минимальный объём продукта в скважине для безубыточной разработки: 111.11111111111111


Средний запас предсказанного сырья по всем регионам ниже необходимого объёма для безубыточной разработки. Если рандомно бурить скважины, то можно предположить, что в среднем будет убыток.

In [18]:
# Функция выбирает скважины лучшие скважины по предсказаниям модели,
# суммирует реальный объём продукта, считает прибыль.
def revenue(predictions_valid, target_valid):
    top_points = predictions_valid.sort_values(ascending=False)[:POINTS_PER_BUDGET].index
    total_product_volume = sum(target_valid[top_points])
    total_revenue = total_product_volume * PRODUCT_PRICE - BUDGET
    return(total_revenue)

**Вывод:** функция для расчёта прибыли по наиболее выгодным шахтам, предсказанным моделью, готова.

## Расчёт прибыли и рисков 

In [19]:
# Функция техникой Bootstrap создаёт 1000 выборок из 500 предсказаний по объёму продукта в скважинах
# определённого региона. Затем функция revenue() отбирает 200 скважин с наибольшими предсказанными
# значениями объёма продукта в скважинах, считает прибыль, после чего это значение заносится в список.
# Расчитывается средняя прибыль, 95% доверительный интервал, риск убытков.
def bootstrap(predictions_valid, target_valid):
    total_revenue_values = []
    for i in range(1000):
        predcitions_subsample = predictions_valid.sample(n=POINTS_TO_EXPLORE, random_state=STATE)
        total_revenue_values.append(revenue(predcitions_subsample, target_valid))
    total_revenue_values = pd.Series(total_revenue_values)
    mean_revenue = total_revenue_values.mean()
    print('Средняя прибыль:', mean_revenue)
    print('95%-ый доверительный интервал для распределения прибыли: ' + \
          f'({total_revenue_values.quantile(.025)}, {total_revenue_values.quantile(.975)})')
    loss_risk = total_revenue_values[total_revenue_values < 0].count() / total_revenue_values.count()
    print(f'Риск убытков: {loss_risk:.1%}')

In [20]:
bootstrap(fr_predictions_valid, fr_target_valid)

Средняя прибыль: 398677700.98736227
95%-ый доверительный интервал для распределения прибыли: (-118269439.91199045, 890419901.0573281)
Риск убытков: 6.3%


In [21]:
bootstrap(sr_predictions_valid, sr_target_valid)

Средняя прибыль: 479515031.02889526
95%-ый доверительный интервал для распределения прибыли: (85925309.62850137, 848824923.1596311)
Риск убытков: 0.4%


In [22]:
bootstrap(tr_predictions_valid, tr_target_valid)

Средняя прибыль: 346328543.7984845
95%-ый доверительный интервал для распределения прибыли: (-175791253.7726245, 845653636.4892236)
Риск убытков: 9.4%


**Вывод:** меньше 2.5% риск убытков наблюдается только во втором регионе, следовательно лучше разрабатывать скважины в нём.

## Общий вывод

**Загрузка и подготовка данных**  
На данном этапе была сделана предобработка данных, разделение исходных датасетов по регионам на выборки с признаками и целевым признаком.  
**Обучение и проверка модели**  
По каждому региону обучена модель линейной регрессии, вычислен средний запас предсказанного сырья, корень среднеквадратической ошибки. Самое низкое значение среднего запаса предсказнного сырья у второго региона, но также наименьшее значение корня среднеквадратической ошибки, что говорит о более точных предиктах модели по данному региону. Сохранены предсказания по валидационной выборке.  
**Подготовка к расчёту прибыли**  
Указаны константы по условию задачи. Написана функция для расчёта прибыли по скважинам, в которых наибольший объём продукта по предсказанию модели.  
**Расчёт прибыли и рисков**  
По каждому региону техникой Bootstrap из выборки с предсказаниями модели по валидационной выборке создано 1000 подвыборок, по каждой из которых расчитана прибыль. Далее рассчитывалась средняя прибыль, доверительный интервал и риск убытков.  
**Заключение**  
Риск убытков ниже 2.5% выявлен только во втором регионе, значит отдать предпочтение разработок скважин следует в данном регионе.